<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
# Imports
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import re
import requests
import pandas as pd

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, SimpleRNN, LSTM

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
# The url where the data is held
url = "https://www.gutenberg.org/files/100/100-0.txt"

# Reading in the data and encoding it
r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]

# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            
# Turn into a DataFrame from a dictionary
df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [3]:
#Shakespeare Data Parsed by Play
print(df_toc.shape)
df_toc.head(8)

(43, 4)


,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...
5,"THE TRAGEDY OF HAMLET, PRINCE OF DENMARK",30365,37051,"THE TRAGEDY OF HAMLET, PRINCE OF DENMARK\r\n\r..."
6,THE FIRST PART OF KING HENRY THE FOURTH,37052,41767,THE FIRST PART OF KING HENRY THE FOURTH\r\n\r\...
7,THE SECOND PART OF KING HENRY THE FOURTH,41768,-100,THE SECOND PART OF KING HENRY THE FOURTH\r\n\r...


In [4]:
# Clean the text
df_toc['text'] = [i.replace('\r\n', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('[', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace(']', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('-', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('_', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('—', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('|', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('`', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('}', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('{', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('(', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace(')', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('\\', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('*', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('/', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('%', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('Æ', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('æ', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('$', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('œ', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace(':', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace(';', ' ') for i in df_toc['text']]
df_toc['text'] = [i.replace('@', ' ') for i in df_toc['text']]

# Look at one of the text values to check work
df_toc['text'][5][:250]

'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK    Contents  ACT I Scene I. Elsinore. A platform before the Castle. Scene II. Elsinore. A room of state in the Castle Scene III. A room in Polonius’s house. Scene IV. The platform. Scene V. A more remote part '

In [5]:
# Remove extra spacing in the text
df_toc['text'] = [re.sub('\s\s+', ' ', i) for i in df_toc.text]

# Look at one of the text values to check work
df_toc['text'][5][:250]

'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK Contents ACT I Scene I. Elsinore. A platform before the Castle. Scene II. Elsinore. A room of state in the Castle Scene III. A room in Polonius’s house. Scene IV. The platform. Scene V. A more remote part of t'

In [6]:
# Character encoding       
# Tried/failed `text = [df_toc['text'][i] for i in range(len(df_toc['text']))]`
text = ' '.join(df_toc['text'])  # Don't need this, done above
chars = list(set(text))

char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

print(f'My corpus contains {len(chars)} unique characters.')

My corpus contains 82 unique characters.


In [7]:
chars

['j',
 'H',
 'r',
 '0',
 'y',
 'x',
 'V',
 'Y',
 '”',
 'N',
 'h',
 '5',
 ',',
 'G',
 'â',
 'F',
 'z',
 '!',
 's',
 'É',
 '1',
 'P',
 'A',
 'p',
 'é',
 '’',
 'B',
 "'",
 'S',
 'u',
 'T',
 'd',
 'ç',
 'g',
 '&',
 '8',
 'i',
 '.',
 't',
 'à',
 'O',
 'E',
 'î',
 'm',
 'w',
 'R',
 'Q',
 'b',
 'q',
 'è',
 'n',
 'U',
 '6',
 'o',
 'C',
 '7',
 'e',
 'f',
 'c',
 '3',
 '2',
 '4',
 'X',
 'Z',
 '9',
 'v',
 'k',
 'D',
 'I',
 'J',
 '“',
 '?',
 '‘',
 'ê',
 ' ',
 'L',
 '"',
 'l',
 'W',
 'M',
 'a',
 'K']

In [13]:
# Create the sequence data
max_len = 150
step = 30

# Encode each character
encoded = [char_int[c] for c in text]
sequences = []  # Each element == max_len
next_chars = []  # One element for each sequence

for i in range(0, len(encoded) - max_len, step):
    sequences.append(encoded[i : i + max_len])
    next_chars.append(encoded[i + max_len])

# Look at the count of sequences
print(f'Length of sequences: {len(sequences)}')

Length of sequences: 464548


In [14]:
# Look at the text
text[0:10]

' AS YOU LI'

In [15]:
# Look at the sequences
sequences[0]

[74,
 22,
 28,
 74,
 7,
 40,
 51,
 74,
 75,
 68,
 81,
 41,
 74,
 68,
 30,
 74,
 67,
 45,
 22,
 79,
 22,
 30,
 68,
 28,
 74,
 21,
 41,
 45,
 28,
 40,
 9,
 22,
 41,
 37,
 74,
 67,
 51,
 81,
 41,
 12,
 74,
 77,
 36,
 65,
 36,
 50,
 33,
 74,
 36,
 50,
 74,
 56,
 5,
 36,
 77,
 56,
 74,
 15,
 45,
 41,
 67,
 41,
 45,
 68,
 54,
 81,
 12,
 74,
 10,
 36,
 18,
 74,
 47,
 2,
 53,
 38,
 10,
 56,
 2,
 12,
 74,
 80,
 50,
 31,
 74,
 29,
 18,
 29,
 2,
 23,
 56,
 2,
 74,
 53,
 57,
 74,
 10,
 36,
 18,
 74,
 31,
 53,
 43,
 36,
 50,
 36,
 53,
 50,
 18,
 74,
 22,
 79,
 68,
 41,
 9,
 28,
 12,
 74,
 77,
 53,
 2,
 31,
 74,
 80,
 38,
 38,
 56,
 50,
 31,
 36,
 50,
 33,
 74,
 53,
 50,
 74,
 38,
 10,
 56,
 74,
 47,
 80,
 50,
 36,
 18,
 10,
 56,
 31,
 74,
 67]

In [16]:
# Look at the next chars
next_chars[0]

29

In [17]:
import numpy as np

# Specify my x(training data) and y (target)
x = np.zeros((len(sequences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1
    y[i, next_chars[i]] = 1

print(x.shape, y.shape)

(464548, 150, 82) (464548, 82)


In [18]:
# Build a single LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(max_len, len(chars)),
               activation='sigmoid'))
model.add(Dropout(0.25))
model.add(LSTM(96, return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(96))
model.add(Dropout(0.25))
model.add(Dense(len(chars), activation='softmax'))

# Compile my model
model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

# Look at the summary of my model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 150, 128)          108032    
_________________________________________________________________
dropout (Dropout)            (None, 150, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 96)           86400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 96)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 96)                74112     
_________________________________________________________________
dropout_2 (Dropout)          (None, 96)                0         
_________________________________________________________________
dense (Dense)                (None, 82)                7

In [1]:
# Fit my model
model.fit(x, y,
          batch_size=64,
          epochs=10)

NameError: name 'model' is not defined

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN